<a href="https://colab.research.google.com/github/ArunK-ML/Project---TripFare-Predicting-Urban-Taxi-Fare-with-Machine-Learning/blob/main/TripFare_ML_ff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚗 **TripFare : Predicting Urban Taxi Fare with Machine Learning**

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Data Collection

df = pd.read_csv("/content/drive/MyDrive/DataSet_For DS/taxi_fare.csv")
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212340,2,2016-03-01 12:15:37,2016-03-01 12:38:53,1,-73.955688,40.782261,1,N,-73.974915,40.751282,1,16.0,0.0,0.5,3.36,0.00,0.3,20.16
212341,2,2016-03-01 12:15:37,2016-03-01 12:38:22,1,-74.002197,40.715042,1,N,-73.976295,40.739609,1,15.5,0.0,0.5,2.00,0.00,0.3,18.30
212342,2,2016-03-01 12:15:37,2016-03-01 12:18:18,1,-73.991173,40.733170,1,N,-73.994324,40.728531,1,4.0,0.0,0.5,0.00,0.00,0.3,4.80
212343,1,2016-03-01 12:15:38,2016-03-01 12:22:47,2,-73.972099,40.786446,1,N,-73.955872,40.779755,2,7.0,0.0,0.5,0.00,0.00,0.3,7.80


In [33]:
df.shape

(212345, 18)

In [34]:
# --------------------------
# 2. Structure & Schema
# --------------------------
print("\n--- Data Types ---")
print(df.dtypes)

print("\n--- Missing Values ---")
print(df.isnull().sum())

print("\n--- Duplicate Rows ---")
print(df.duplicated().sum())


--- Data Types ---
VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
pickup_longitude         float64
pickup_latitude          float64
RatecodeID                 int64
store_and_fwd_flag        object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

--- Missing Values ---
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
pickup_longitude         0
pickup_latitude          0
RatecodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                   

In [35]:
# Change dtype

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

In [36]:
# Haversine formula function
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Apply to your DataFrame
df["trip_distance_km"] = haversine_distance(
    df["pickup_latitude"],
    df["pickup_longitude"],
    df["dropoff_latitude"],
    df["dropoff_longitude"]
)
df = df.drop(columns=["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"])

In [37]:
import pytz

# Ensure pickup_hour is extracted
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Extract AM/PM
df['pickup_am_pm'] = df['tpep_pickup_datetime'].dt.strftime('%p')

# Binary flag for night trips
df['is_night'] = df['pickup_hour'].apply(
    lambda x: 1 if (x >= 22 or x <= 5) else 0
)

# Calculate trip duration in minutes
df['trip_duration_min'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Or in hours
df['trip_duration_hr'] = df['trip_duration_min'] / 60

# Drop the columns
df = df.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [38]:
df

,VendorID,passenger_count,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_distance_km,pickup_hour,pickup_am_pm,is_night,trip_duration_min,trip_duration_hr
0,1,1,1,N,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,3.138096,0,AM,1,7.916667,0.131944
1,1,1,1,N,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,4.303331,0,AM,1,11.100000,0.185000
2,2,2,1,N,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,16.600142,0,AM,1,31.100000,0.518333
3,2,3,1,N,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,9.046547,0,AM,1,0.000000,0.000000
4,2,5,3,N,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,20.400022,0,AM,1,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212340,2,1,1,N,1,16.0,0.0,0.5,3.36,0.00,0.3,20.16,3.806279,12,PM,0,23.266667,0.387778
212341,2,1,1,N,1,15.5,0.0,0.5,2.00,0.00,0.3,18.30,3.496576,12,PM,0,22.750000,0.379167
212342,2,1,1,N,1,4.0,0.0,0.5,0.00,0.00,0.3,4.80,0.580119,12,PM,0,2.683333,0.044722
212343,1,2,1,N,2,7.0,0.0,0.5,0.00,0.00,0.3,7.80,1.555737,12,PM,0,7.150000,0.119167


In [39]:
# Count zeros and negative values in each column

# Select only numeric columns
numeric_df = df.select_dtypes(include=np.number)

# Count zeros in each column
zero_count = (numeric_df == 0).sum()

# Count negative values in each column
negative_count = (numeric_df < 0).sum()

# Combine results
result = pd.DataFrame({
    'Zero_Count': zero_count,
    'Negative_Count': negative_count
})

print(result)

                       Zero_Count  Negative_Count
VendorID                        0               0
passenger_count                 9               0
RatecodeID                      0               0
payment_type                    0               0
fare_amount                    50             106
extra                      191930              19
mta_tax                       791             101
tip_amount                  73542               4
tolls_amount               200726               0
improvement_surcharge          45             106
total_amount                   38             106
trip_distance_km             2982               0
pickup_hour                  7079               0
is_night                   191030               0
trip_duration_min             130               1
trip_duration_hr              130               1


In [40]:
# Drop rows where passenger_count == 0
df = df[df['passenger_count'] != 0]

In [41]:
# 1️⃣ Drop rows where passenger_count == 0
df = df[df['passenger_count'] != 0]

# 2️⃣ Get numeric column names
numeric_cols = df.select_dtypes(include=['number']).columns

# 3️⃣ Convert all numeric values to positive
df[numeric_cols] = df[numeric_cols].abs()

In [42]:
# ---- 1️⃣ Count zeros & negatives in each column ----
zero_count = (numeric_df == 0).sum()
negative_count = (numeric_df < 0).sum()

column_summary = pd.DataFrame({
    'Zero_Count': zero_count,
    'Negative_Count': negative_count
})

print("🔹 Column-wise Zero & Negative Counts")
print(column_summary, "\n")

# ---- 2️⃣ Find rows with max zero values ----
row_zero_counts = (numeric_df == 0).sum(axis=1)  # Count 0's per row
max_zeros = row_zero_counts.max()
rows_with_max_zeros = numeric_df[row_zero_counts == max_zeros]

print(f"🔹 Row(s) with Maximum Zero Values ({max_zeros} zeros):")
print(rows_with_max_zeros)

🔹 Column-wise Zero & Negative Counts
                       Zero_Count  Negative_Count
VendorID                        0               0
passenger_count                 9               0
RatecodeID                      0               0
payment_type                    0               0
fare_amount                    50             106
extra                      191930              19
mta_tax                       791             101
tip_amount                  73542               4
tolls_amount               200726               0
improvement_surcharge          45             106
total_amount                   38             106
trip_distance_km             2982               0
pickup_hour                  7079               0
is_night                   191030               0
trip_duration_min             130               1
trip_duration_hr              130               1 

🔹 Row(s) with Maximum Zero Values (11 zeros):
        VendorID  passenger_count  RatecodeID  payment_type  fare

In [53]:
df = df.drop(index=159884)

In [54]:
# Select only numeric columns
numeric_df = df.select_dtypes(include=np.number)

# Count zeros in each column
zero_count = (numeric_df == 0).sum()

# Count negative values in each column
negative_count = (numeric_df < 0).sum()

# Combine results
result = pd.DataFrame({
    'Zero_Count': zero_count,
    'Negative_Count': negative_count
})

print(result)

                       Zero_Count  Negative_Count
VendorID                        0               0
passenger_count                 0               0
RatecodeID                      0               0
payment_type                    0               0
fare_amount                    47               0
extra                      191921               0
mta_tax                       784               0
tip_amount                  73534               0
tolls_amount               200717               0
improvement_surcharge          41               0
total_amount                   36               0
trip_distance_km             2980               0
pickup_hour                  7079               0
is_night                   191022               0
trip_duration_min             129               0
trip_duration_hr              129               0


In [43]:
# --------------------------
# 3. Distribution & Skewness
# --------------------------

# Step 1: Select numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns

# Step 2: Loop through numeric columns
for col in numeric_cols:
    skew_val = df[col].skew()
    kurt_val = df[col].kurtosis()

    print(f"\nColumn: {col}")
    print(f"  Skewness: {skew_val:.3f}")
    print(f"  Kurtosis: {kurt_val:.3f}")

    # Step 3: Apply normality check thresholds
    if abs(skew_val) < 0.5 and abs(kurt_val) < 3:
        print("  ✅ Looks approximately normal")
    else:
        print("  ⚠ Not normal (may need transformation)")


Column: VendorID
  Skewness: -0.917
  Kurtosis: -1.159
  ⚠ Not normal (may need transformation)

Column: passenger_count
  Skewness: 1.887
  Kurtosis: 2.148
  ⚠ Not normal (may need transformation)

Column: RatecodeID
  Skewness: 128.393
  Kurtosis: 36721.744
  ⚠ Not normal (may need transformation)

Column: payment_type
  Skewness: 0.923
  Kurtosis: -0.428
  ⚠ Not normal (may need transformation)

Column: fare_amount
  Skewness: 6.091
  Kurtosis: 203.005
  ⚠ Not normal (may need transformation)

Column: extra
  Skewness: 2.755
  Kurtosis: 5.672
  ⚠ Not normal (may need transformation)

Column: mta_tax
  Skewness: -16.355
  Kurtosis: 265.502
  ⚠ Not normal (may need transformation)

Column: tip_amount
  Skewness: 7.664
  Kurtosis: 351.965
  ⚠ Not normal (may need transformation)

Column: tolls_amount
  Skewness: 164.741
  Kurtosis: 53067.211
  ⚠ Not normal (may need transformation)

Column: improvement_surcharge
  Skewness: -71.082
  Kurtosis: 5050.738
  ⚠ Not normal (may need transfo

In [44]:
# Transformation

# Methods (lambda values for Box-Cox)
methods = [0, 0.5, -0.5, -1, 1, 2, -2]

# Loop through numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns

for col in numeric_cols:
    print(f"\n=== Column: {col} ===")

    # Box-Cox requires positive values
    if (df[col] <= 0).any():
        print("❌ Skipped (contains zero or negative values)")
        continue

    for j in methods:
        # Apply Box-Cox
        transformed = boxcox(df[col], lmbda=j)

        # Calculate skewness and kurtosis
        skew_val = pd.Series(transformed).skew()
        kurt_val = pd.Series(transformed).kurtosis()

        print(f"λ = {j}:  skew = {skew_val:.3f}, kurtosis = {kurt_val:.3f}")


=== Column: VendorID ===
λ = 0:  skew = -0.917, kurtosis = -1.159
λ = 0.5:  skew = -0.917, kurtosis = -1.159
λ = -0.5:  skew = -0.917, kurtosis = -1.159
λ = -1:  skew = -0.917, kurtosis = -1.159
λ = 1:  skew = -0.917, kurtosis = -1.159
λ = 2:  skew = -0.917, kurtosis = -1.159
λ = -2:  skew = -0.917, kurtosis = -1.159

=== Column: passenger_count ===
λ = 0:  skew = 1.429, kurtosis = 0.518
λ = 0.5:  skew = 1.654, kurtosis = 1.291
λ = -0.5:  skew = 1.239, kurtosis = -0.104
λ = -1:  skew = 1.102, kurtosis = -0.547
λ = 1:  skew = 1.887, kurtosis = 2.148
λ = 2:  skew = 2.294, kurtosis = 3.922
λ = -2:  skew = 0.962, kurtosis = -0.993

=== Column: RatecodeID ===
λ = 0:  skew = 8.017, kurtosis = 76.100
λ = 0.5:  skew = 13.093, kurtosis = 541.933
λ = -0.5:  skew = 7.164, kurtosis = 54.283
λ = -1:  skew = 6.770, kurtosis = 46.106
λ = 1:  skew = 128.393, kurtosis = 36721.744
λ = 2:  skew = 458.352, kurtosis = 210831.420
λ = -2:  skew = 6.478, kurtosis = 40.458

=== Column: payment_type ===
λ = 0:

In [45]:
numeric_cols = df.select_dtypes(include=np.number).columns

for col in numeric_cols:
    plt.figure(figsize=(5,3))
    sns.histplot(df[col], kde=True)-
    plt.title(f"Distribution of {col}")
    plt.show()
    print(f"{col} Skewness: {skew(df[col].dropna())}")

SyntaxError: invalid syntax (ipython-input-3996386587.py, line 5)

In [ ]:
# --------------------------
# 4. Outliers Detection (Numeric Columns)
# --------------------------
numeric_cols = df.select_dtypes(include=np.number).columns
for col in numeric_cols:
    plt.figure(figsize=(5,3))
    sns.boxplot(x=df[col])
    plt.title(f"Outliers in {col}")
    plt.show()

# ** Preprocessing**

**Feature Engineering**

In [ ]:
import pytz

# Ensure pickup_hour is extracted
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Extract AM/PM
df['pickup_am_pm'] = df['tpep_pickup_datetime'].dt.strftime('%p')

# Binary flag for night trips
df['is_night'] = df['pickup_hour'].apply(
    lambda x: 1 if (x >= 22 or x <= 5) else 0
)

# Step 1: Localize as UTC
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].dt.tz_localize('UTC')

# Step 2: Convert to US/Eastern (EDT/EST based on date)
df['tpep_pickup_datetime_(EDT)'] = df['tpep_pickup_datetime'].dt.tz_convert('US/Eastern')

# Step 1: Localize as UTC
df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].dt.tz_localize('UTC')

# Step 2: Convert to US/Eastern (EDT/EST based on date)
df['tpep_dropoff_datetime_(EDT)'] = df['tpep_dropoff_datetime'].dt.tz_convert('US/Eastern')



In [ ]:

df_copy = df.copy()

# Encode Categorical Data

from sklearn.preprocessing import LabelEncoder
code = LabelEncoder()

for i in df_copy.select_dtypes(include="object").columns:
  df_copy[i] = code.fit_transform(df_copy[i])


# -----------------------------------------------------------------------------
#                               Correlation Test
# ---------------------------------------------------------------------------

import seaborn as sns
import matplotlib.pyplot as plt

# ----------------------------------------
# Calculate correlation for Trip Fare
# ----------------------------------------
corr = df_copy.corr()

# Plot heatmap
plt.figure(figsize=(16, 14))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Trip Fare Heatmap")
plt.show()



# ** Exploratory Data Analysis (EDA)**

In [ ]:
#  Fare vs. Distance: Exploratory Data Analysis

# ----------------------------
# Step 3: Scatter Plot (Fare vs. Distance)
# ----------------------------
plt.figure(figsize=(10, 6))
plt.hexbin(
    df['trip_distance_km'],
    df['fare_amount'],
    gridsize=50,
    cmap='viridis',
    bins='log'
)
plt.colorbar(label='Log(Count)')
plt.title('Fare Amount vs. Trip Distance (Hexbin)')
plt.xlabel('Trip Distance (km)')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------
# Plot: Fare vs. Passenger Count
# ----------------------------
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='passenger_count', y='fare_amount')
plt.title('Fare Amount vs. Passenger Count')
plt.xlabel('Passenger Count')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()

# **Hypothesis testing**

In [ ]:
df.to_csv("taxi_fare_cleaned.csv")

In [ ]:
df.columns

In [ ]:
catergory = [
       'passenger_count', 'RatecodeID','store_and_fwd_flag',
       'payment_type','improvement_surcharge', 'total_amount',
        'pickup_am_pm', 'is_night']

continous = ['VendorID',
            'fare_amount', 'extra', 'mta_tax', 'tip_amount','tolls_amount',
             'trip_distance_km','pickup_hour'
             ]

# **Data Transformation**

In [ ]:

from scipy import stats

#continious vs continious
def two_sample(d1,d2):
  t=0
  f=0
  for i in  range(31):
    sample1=d1.sample(frac=0.03)
    sample2=d2.sample(frac=0.03)
    t_test,p_value=stats.ttest_ind(sample1,sample2)
    if p_value < 0.1:
      f=f+1
    else:
      t=t+1
  if t>f:
    return True
  else:
    return False


In [ ]:


#cat vs cat
def chisquare_cat_vs_cat(d1,d2):
    return True if stats.chi2_contingency(pd.crosstab(d1,d2))[1] < 0.1 else False

In [ ]:
#continious(d1) vs cat(d2)
def annova_test(d1,d2):
    group= df[d2].unique()
    data = {}
    for i in group:
        # Check if the category has more than one data point
        if len(df[d1][df[d2]==i]) > 1:
            data[i]=df[d1][df[d2]==i]
    # Check if there are at least two groups with more than one data point
    if len(data) < 2:
        return False  # Cannot perform ANOVA with less than two groups
    f_value,p_value=stats.f_oneway(*[i for i in data.values()])
    return False if p_value < 0.1 else True


In [ ]:

final = {}
for i in df.columns:
    final[i]={}
    for j in df.columns:
        if (i in continous) and (j in continous):
            result=two_sample(df[i],df[j])
        elif (i in continous) and (j in catergory):
            result=annova_test(i,j)
        elif (i in catergory) and (j in continous):
            result=annova_test(j,i)
        elif (i in catergory) and (j in catergory):
            result=chisquare_cat_vs_cat(df[i],df[j])
        if result:
            final[i][j]=1
        else:
            final[i][j]=0

In [ ]:
final_df = pd.DataFrame(final)
final_df

In [ ]:
import seaborn as sns
sns.heatmap(final_df,annot=True,cmap="coolwarm")

# **Train the model ML**

In [ ]:
df_copy.columns

In [ ]:
df_copy.info()

In [ ]:
y = df_copy['total_amount']
x = df_copy.drop('total_amount',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,r2_score

# Drop datetime columns before training
x_train = x_train.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'tpep_pickup_datetime_(EDT)', 'tpep_dropoff_datetime_(EDT)'])
x_test = x_test.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'tpep_pickup_datetime_(EDT)', 'tpep_dropoff_datetime_(EDT)'])

model = RandomForestRegressor().fit(x_train,y_train)
y_pred = model.predict(x_test)
print(mean_absolute_error(y_test,y_pred))
print(mean_absolute_percentage_error(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

In [ ]:
from scipy.stats import boxcox

# Transformation
method = [0, 0.5, -0.5, -1, 1, 2, -2]
numeric_cols = df.select_dtypes(include=np.number).columns

for col in numeric_cols:
    print(f"\n--- Transformation for {col} ---")
    for j in method:
        try:
            # Apply Box-Cox transformation
            transformed_data, _ = boxcox(df[col] + 1, lmbda=j) # Added 1 to handle non-positive values

            # Calculate skewness and kurtosis of transformed data
            skewness = pd.DataFrame(transformed_data).skew().values[0]
            kurtosis = pd.DataFrame(transformed_data).kurtosis().values[0]

            print(f"Lambda = {j}")
            print(f"Skewness: {skewness:.3f}")
            print(f"Kurtosis: {kurtosis:.3f}")
            print("----------------------------------------------------------------")
        except ValueError as e:
            print(f"Could not apply Box-Cox with lambda={j} to {col}: {e}")
            print("----------------------------------------------------------------")